In [1]:
pip install nums_from_string

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import requests
import datetime
import nums_from_string
import csv
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [3]:
data = []
cities=['Mumbai', 'Goa']

In [4]:
today = datetime.datetime.today().strftime ('%Y-%m-%d')
def get_date_posted(ago_count):
  Previous_Date = datetime.datetime.today() - datetime.timedelta(days=ago_count)
  previous_d_for = Previous_Date.strftime ('%d/%m/%Y')
  return previous_d_for

def scrape(city):
  for i in range(1,20):

    url = "https://www.magicbricks.com/property-for-rent/residential-commercial-real-estate?proptype=Multistorey-Apartment,Builder-Floor-Apartment,Penthouse,Studio-Apartment,Residential-House,Villa,Residential-Plot,Commercial-Office-Space,Office-ITPark-SEZ,Commercial-Shop,Commercial-Showroom,Commercial-Land,Industrial-Land,Warehouse/-Godown,Industrial-Building,Industrial-Shed&page="+str(i)+"&cityName="+city+"&language=en"
    response = requests.get(url)
    response = response.content
    soup = BeautifulSoup(response, 'html.parser')
    cards = soup.find_all('div', class_='mb-srp__card')

    locality = None
    furnishing = None
    carpet_area = None
    availability = None
    facing = None
    floor = None
    tenant = None
    car_parking = None
    society = None
    desc = None
    date_posted = None

    for card in cards:
      try:
        posted_date = card.find(class_="mb-srp__card__photo__fig--post").text
        if 'today' in posted_date or 'Today' in posted_date:
          date_posted = get_date_posted(0)
        elif 'ago' in posted_date:
          ago_date_count = int(nums_from_string.get_nums(posted_date)[0])
          if 'days' in posted_date:
              date_posted = get_date_posted(ago_date_count)
          elif 'weeks' in posted_date:
              date_posted = get_date_posted((ago_date_count*7))
          else:
              date_posted = get_date_posted((ago_date_count*30))
        elif 'yesterday' in posted_date or 'Yesterday' in posted_date:
            date_posted = get_date_posted(1)
        else:
            date_posted = posted_date.replace('Posted: ','')
            
      except:
          posted_date = None
      try: 
          link = card.find('a', class_='mb-srp__card__society--name')['href']
      except:
          link = None
      try: 
          owner = card.find('div', class_='mb-srp__card__ads--name').text
      except:
          owner = None
      bhks = card.find('h2', class_='mb-srp__card--title').text
      prop_locality = bhks.split(', ')
      bhk = prop_locality[0][0:5]
      del prop_locality[0]
      try:
        locality = prop_locality[0]
      except:
        locality = city
      childs = card.find('div', class_='mb-srp__card__summary__list')
      description = card.find('div', class_='mb-srp__card--desc')
      try: 
          desc = description.find('p').text.replace('"', '')
      except:
          desc = None
      rent = card.find('div', class_='mb-srp__card__price--amount').text.replace('₹', '')
      for elem in childs.contents:
        try:
          if elem.get('data-summary') == 'facing':
            facing = elem.text.replace('facing', '')
        except:
          facing= None
        try:
          if elem.get('data-summary') == 'tenent-preffered':
            tenant = elem.text.replace('Tenant Preferred', '')
        except:
          tenant= None
        try:
          if elem.get('data-summary') == 'floor':
            floor = elem.text.replace('Floor', '')
        except:
          floor= None
        try:
          if elem.get('data-summary') == 'status':
            availability = elem.text.replace('Availability', '')
        except:
          availability= None
        try:
          if elem.get('data-summary') == 'society':
            society = elem.text.replace('Society', '')
        except:
          society= None
        try:    
          if elem.get('data-summary') == 'furnishing':
            furnishing = elem.text.replace('Furnishing', '')  
        except:
          furnishing = None
        try: 
          if elem.get('data-summary') == 'parking':
            car_parking = elem.text.replace('Car Parking', '')
        except:
          car_parking = None
        try:
          if elem.get('data-summary') == 'carpet-area':
            carpet_area = elem.text.replace('Carpet Area', '')
        except:
          carpet_area= None
        try:
          if ('Owner' in owner) or ('Agent' in owner):
            data.append([date_posted, link, owner, bhk, locality, rent, furnishing, car_parking, facing, availability, floor, tenant, carpet_area, desc])
        except:
          data.append([date_posted, link, owner, bhk, locality, rent, furnishing, car_parking, facing, availability, floor, tenant, carpet_area, desc])
  df = pd.DataFrame(data, columns=['Date Posted', 'Link', 'Owner', 'BHK', 'Locality', 'Rent', 'Furnishing', 'Car Parking', 'Facing', 'Availability', 'Floor', 'Tenant Preferred', 'Carpet Area', 'Description'])
  df.to_csv('Data_{0}.csv'.format(str(city)), index=False)
  df.drop_duplicates(subset='Link', keep='first', inplace=True)
  df.insert(loc=0, column='No', value=np.arange(len(df)))
  try:
    df.drop(index=df.index[0], axis=0, inplace=True)
  except:
    df = df.fillna(0)
  df.to_csv('{0}_{1}_rent.csv'.format(str(city), today), index=False)

  # Function to convert a CSV to JSON
  def make_json(csvFilePath, jsonFilePath):
    
    data = {}
    
    with open(csvFilePath, encoding='utf-8') as csvf:
      csvReader = csv.DictReader(csvf)

      for rows in csvReader:

        key = rows['No']
        data[key] = rows

    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
      jsonf.write(json.dumps(data, indent=4))

  csvFilePath = r'{0}_{1}_rent.csv'.format(str(city), today)
  jsonFilePath = r'{0}_{1}_rent.json'.format(str(city), today)

  make_json(csvFilePath, jsonFilePath)

In [5]:
for city in cities:
  scrape(city)